# Import Libraries

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

# Azure ML Setup

In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

# choose a name for experiment
experiment_name = 'customer-churn-automl-experiment'

experiment=Experiment(ws, experiment_name)

Ready to use Azure ML 1.0.83 to work with sbirkamlws


# Create Compute Target

In [3]:
# Create or attach compute target
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your AmlCompute cluster.
amlcompute_cluster_name = "cpu-cluster-1"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'cpu-cluster-1':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_DS12_V2", # for GPU, use "STANDARD_NC6"
                                                                #vm_priority = 'lowpriority', # optional
                                                                max_nodes = 6)

    # Create the cluster.
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

# For a more detailed view of current AmlCompute status, use get_status().

Creating a new compute target...
Checking cluster status...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [4]:
# Get metrics for evaluation of classification models
from azureml.train.automl.utilities import get_primary_metrics

get_primary_metrics('classification')

['precision_score_weighted',
 'average_precision_score_weighted',
 'norm_macro_recall',
 'AUC_weighted',
 'accuracy']

# Configure and Start Auto ML Run

In [5]:
automl_settings = {
    "primary_metric": 'AUC_weighted',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 5,
    "experiment_timeout_hours": 0.2,
    "verbosity": logging.INFO,
    "featurization": "auto"
}

# Load the data
dataset_name = 'customer-churn'

dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

training_data, validation_data = dataset.random_split(percentage=0.8, seed=42)


label_column_name = "Churn"

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target = compute_target,
                             training_data = training_data,
                             validation_data = validation_data,
                             label_column_name = label_column_name,
                             **automl_settings
                            )

In [6]:
remote_run = experiment.submit(automl_config, show_output = False)

In [ ]:
remote_run.wait_for_completion(show_output=False)

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…